<a href="https://colab.research.google.com/github/Man-bearpig/ID2223/blob/main/lab2/training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  7 08:11:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    54W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,342 kB

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-dlwe01ob
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-dlwe01ob
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 5.0 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5885055 sha256=f7ae6509bb64fb5648b53365d196f80ee3267904a309f722a028f709f423f742
  Stored in directory: /tmp/pip-ephem-wheel-cache-i3s1w7k7/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

In [ ]:
!pip install hopsworks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 120 kB 8.7 MB/s 
     |████████████████████████████████| 50 kB 8.7 MB/s 
     |████████████████████████████████| 132 kB 45.3 MB/s 
     |████████████████████████████████| 45 kB 4.2 MB/s 
     |████████████████████████████████| 68 kB 7.9 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 4.9 MB 72.4 MB/s 
     |████████████████████████████████| 42 kB 2.0 MB/s 
     |████████████████████████████████| 2.8 MB 72.2 MB/s 
     |████████████████████████████████| 2.6 MB 64.8 MB/s 
     |████████████████████████████████| 67 kB 8.0 MB/s 
     |████████████████████████████████| 109 kB 93.7 MB/s 
     |████████████████████████████████| 1.6 MB 82.8 MB/s 
     |████████████████████████████████| 555 kB 74.4 MB/s 
     |████████████████████████████████| 79 kB 11.3 MB/s 
     |████████████████████████████████| 10.2 

In [ ]:

from huggingface_hub import notebook_login

notebook_login()


Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:

import hopsworks
project = hopsworks.login()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/4264


In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")


In [ ]:

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967M [00:00<?, ?B/s]

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


Cloning https://huggingface.co/Manbearpig01/whisper-small-hi into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.43k/922M [00:00<?, ?B/s]

Download file runs/Dec05_20-56-52_2acf91097346/1670273821.507304/events.out.tfevents.1670273821.2acf91097346.1…

Download file runs/Dec05_19-57-33_2acf91097346/events.out.tfevents.1670270620.2acf91097346.77.0: 100%|########…

Download file runs/Dec03_00-23-29_9e676dfb6040/1670027023.6840463/events.out.tfevents.1670027023.9e676dfb6040.…

...

Clean file runs/Dec05_20-56-52_2acf91097346/events.out.tfevents.1670273821.2acf91097346.108814.0:   9%|9      …

Download file training_args.bin: 100%|##########| 3.42k/3.42k [00:00<?, ?B/s]

Clean file training_args.bin:  29%|##9       | 1.00k/3.42k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/922M [00:00<?, ?B/s]

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [ ]:
processor.save_pretrained(training_args.output_dir)

Feature extractor saved in ./whisper-small-hi/preprocessor_config.json
tokenizer config file saved in ./whisper-small-hi/tokenizer_config.json
Special tokens file saved in ./whisper-small-hi/special_tokens_map.json
added tokens file saved in ./whisper-small-hi/added_tokens.json


In [ ]:
trainer.train()


***** Running training *****
  Num examples = 12360
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4000
  Number of trainable parameters = 241734912
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.137800,0.295291,21.416472
2000,0.047500,0.291312,20.327499
3000,0.018700,0.302585,19.899976
4000,0.004300,0.327523,19.684870


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

TrainOutput(global_step=4000, training_loss=0.18868207383342087, metrics={'train_runtime': 16589.3021, 'train_samples_per_second': 3.858, 'train_steps_per_second': 0.241, 'total_flos': 1.84579222044672e+19, 'train_loss': 0.18868207383342087, 'epoch': 5.17})

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "hi",
    "model_name": "Whisper Small Hi - Swedish",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}


In [ ]:
trainer.push_to_hub(**kwargs)

Saving model checkpoint to ./whisper-small-hi
Configuration saved in ./whisper-small-hi/config.json
Model weights saved in ./whisper-small-hi/pytorch_model.bin
Feature extractor saved in ./whisper-small-hi/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/922M [00:00<?, ?B/s]

Upload file runs/Dec07_08-57-26_c5424e0d7480/events.out.tfevents.1670403801.c5424e0d7480.114.0:  11%|#        …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Manbearpig01/whisper-small-hi
   344a74a..302e7b7  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Manbearpig01/whisper-small-hi
   344a74a..302e7b7  main -> main

To https://huggingface.co/Manbearpig01/whisper-small-hi
   302e7b7..a8916df  main -> main

   302e7b7..a8916df  main -> main



'https://huggingface.co/Manbearpig01/whisper-small-hi/commit/302e7b7f7af82cd076f86c6ea549e89828c82025'

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="jdowling/whisper-small-hi")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small Swedish",
    description="Realtime demo for Swedish speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()


In [ ]:
# -*- coding: utf-8 -*-
"""app.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1SLY6vFgJGYJxXCiJWtRo3Qxag5r_Y4K7
"""

!pip install transformers
!pip install pytube

!pip install gradio

import os
import gradio as gr
from transformers import pipeline
from pytube import YouTube


pipe = pipeline(model="Manbearpig01/whisper-small-hi")  # change to "your-username/the-name-you-picked" 加


def yt(link):
    yt = YouTube(link)
    stream = yt.streams.filter(only_audio=True)[0]
    stream.download(filename="audio.mp3")
    text = pipe("audio.mp3")["text"]
    return text

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

demo = gr.Blocks()


iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small Swedish-Microphone",
    description="Realtime demo for Swedish speech recognition using a fine-tuned Whisper small model. An audio for recognize.",
)

yt = gr.Interface(
    fn=yt,
    inputs=[gr.inputs.Textbox(lines=1, label="Youtube URL")],
    outputs=["html", "text"],
    title="Whisper Small Swedish-Youtube",
    description="Realtime demo for Swedish speech recognition using a fine-tuned Whisper small model. A Youtube URL for recognize."

)

with demo:
    gr.TabbedInterface([iface, yt], ["Transcribe Audio", "Transcribe YouTube"])

demo.launch(enable_queue=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Manbearpig01--whisper-small-hi/snapshots/a8916df1f1bf0d53e2f026c448e3eda85beb59cd/config.json
Model config WhisperConfig {
  "_name_or_path": "Manbearpig01/whisper-small-hi",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 50257,
  "forced_decoder_ids": null,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 448,
  "max_source_positions": 1500,
  "max_target_positions": 448,
  "model_type": "wh

Downloading:   0%|          | 0.00/967M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--Manbearpig01--whisper-small-hi/snapshots/a8916df1f1bf0d53e2f026c448e3eda85beb59cd/pytorch_model.bin
All model checkpoint weights were used when initializing WhisperForConditionalGeneration.

All the weights of WhisperForConditionalGeneration were initialized from the model checkpoint at Manbearpig01/whisper-small-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use WhisperForConditionalGeneration for predictions without further training.


...

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Manbearpig01--whisper-small-hi/snapshots/a8916df1f1bf0d53e2f026c448e3eda85beb59cd/vocab.json
loading file tokenizer.json from cache at None
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--Manbearpig01--whisper-small-hi/snapshots/a8916df1f1bf0d53e2f026c448e3eda85beb59cd/merges.txt
loading file normalizer.json from cache at /root/.cache/huggingface/hub/models--Manbearpig01--whisper-small-hi/snapshots/a8916df1f1bf0d53e2f026c448e3eda85beb59cd/normalizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--Manbearpig01--whisper-small-hi/snapshots/a8916df1f1bf0d53e2f026c448e3eda85beb59cd/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--Manbearpig01--whisper-small-hi/snapshots/a8916df1f1bf0d53e2f026c448e3eda85beb59cd/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/h

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
   

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://b9c01e26eb91b73e.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')